<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Prompt Engineering for Quality Decisions

Decision Intelligence applied in this module:  
* Optimizing prompts for logic, reasoning and decisions
* Examples of applying decision intelligence techniques in prompts and their output effect 
* Prompt techniques such as Chain of Thought to improve the quality of the decision reasoning and outcome 


In this module, the Semantic Kernel ability to chat completion experience will be used to optimize system prompts (personas) and instructive prompts. No new specific Semantic Kernel functionality will be introduced. This module will focus on introducing decision intelligence prompting techniques.  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [2]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0

In [3]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.32.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.0 Microsoft.SemanticKernel, 1.32.0

Using Azure OpenAI Service


### Step 2 - Chain of Thought Reasoning for Decision Intelligence 

> "The happiness of your life depends upon the quality of your thoughts." 
>
> -- <cite>Marcus Aurelius (Roman emperor, philosopher)</cite>  

One of the most effective techniques to improve logic, reasoning and overall decision-making for most LLMs is to use "Chain of Thought" (CoT) techniques. The "Chain of Thought" approach helps by breaking down complex tasks into smaller steps, making it easier to solve problems without missing important details. It improves accuracy because each step is made clear to the LLM and helps avoid mistakes. This method also makes the thought process easier to understand and explain, allowing for easier corrections when something goes wrong. The approach is useful for handling difficult concepts and multi-step problems like math or logic, making things clearer and more manageable.

The name of the "Chain of Thought" technique has been recently popularized by Generative AI. However, this technique is not new nor is it specific only to Generative AI models. Breaking down complex tasks into simpler more approachable steps and organizing thoughts has been used by decision-makers, professional services organizations and management consulting companies for quite some time. Prior to Generative AI, the concepts have been known as: "structured thinking", "MECE framework" (Mutually Exclusive, Collectively Exhaustive), and "hypothesis-driven problem-solving".  

First, approach the problem with a simple Decision prompt technique. The system decisoin prompt will be set to basic & clear decision assistant persona instructions. While this decision prompt is simple, it is still quite effective to instruct the LLM model on how to approach the decision problem. 

In [13]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Provide structured, logical, and comprehensive advice.
""";

// Simple instruction prompt to plan retirement
var puzzlePrompt = """
A farmer is on one side of a river with a wolf, a goat, and a cabbage. 
When he is crossing the river in a boat, he can only take one item with him at a time. 
The wolf will eat the goat if left alone together, and the goat will eat the cabbage if left alone together. 
How can the farmer transport the goat across the river without it being eaten?
""";

// Build Chat History with system prompt and the puzzle prompt
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemDecisionPrompt);
chatHistory.AddUserMessage(puzzlePrompt);

// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var decisionResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory, openAIPromptExecutionSettings))
{
    decisionResponse += content;
    Console.Write(content);
}

To solve this problem, the farmer must carefully plan the sequence of crossings to ensure that the wolf, goat, and cabbage are never left in a situation where one can eat the other. Here's a structured step-by-step solution:

---

### **Step 1: Transport the goat across the river.**
- The farmer takes the goat across the river first.
- Now, the wolf and the cabbage are left on the original side, and they are safe together because the wolf does not eat the cabbage.

---

### **Step 2: Return to the original side.**
- The farmer goes back to the original side alone, leaving the goat on the far side of the river.

---

### **Step 3: Transport the wolf across the river.**
- The farmer takes the wolf across the river to the far side.
- However, the goat and the wolf cannot be left together, so the farmer brings the goat back to the original side.

---

### **Step 4: Transport the cabbage across the river.**
- The farmer takes the cabbage across the river to the far side, where the wolf is n

Let's approach the same problem with a much more sophisticated "Chain of Thought" (CoT) system prompt to break the problem down and think about it more in depth. Notice that the prompt below is much more detailed in how the "decision intelligence assistant" should approach the problem, highlighting things it should or should not consider and the way it should arrive at the final answer. 

Chain of Thought (CoT) prompt instructions will vary depending on how the problem needs to be approached. It can include very specific decision framework instructions, systematic decision heuristics or management consulting best practices. In general, these detailed prompts result in the LLM providing more detail and usually better outcomes. However, there are a few drawbacks to overusing Chain of Thought (CoT):
* A very long and detailed Chain of Thought (CoT) can confuse the LLM model, especially if the LLM model is smaller (i.e. GPT-4o-mini or Phi-4 etc.)
* GenAI LLM models can hallucinate not just on output of answers. They can also hallucinate the Chain of Thought (CoT) they are describing in the final answer! This is a very sneaky way of potentially providing a confident answer paired with a confident approach (Chain of Thought) that the LLM may not even be using! 

In [14]:
// Set the system prompt to behave like a decision intelligence assistant (persona)
var systemPromptChainOfThought = """
You are a Decision Intelligence assistant designed to think through problems step-by-step using Chain-of-Thought (COT) prompting. 
Before providing any answer, you must: 

1) Understand the Problem: Carefully read and understand the user's question or request. 

2) Break Down the Reasoning Process: Outline the steps required to solve the problem or respond to the request logically and sequentially. Think aloud and describe each step in detail. 
Explain Each Step: Provide reasoning or calculations for each step, explaining how you arrive at each part of your answer. 
Provide structured, logical, and comprehensive advice. 

3) Review the Thought Process: Double-check the reasoning for errors or gaps before finalizing your response. 
Always aim to make your thought process transparent and logical.

4) Arrive at the Final Answer: Only after completing all steps, provide the final answer or solution. 

5) Communicate the final decision using the Minto Pyramid Principle.
""";

// Simple instruction prompt to plan retirement
var puzzlePrompt = """
A farmer is on one side of a river with a wolf, a goat, and a cabbage. 
When he is crossing the river in a boat, he can only take one item with him at a time. 
The wolf will eat the goat if left alone together, and the goat will eat the cabbage if left alone together. 
How can the farmer transport the goat across the river without it being eaten?
""";

// Create a new chat history object using the new Chain of Thought system prompt
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPromptChainOfThought);
chatHistory.AddUserMessage(puzzlePrompt);

// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var decisionResponseChainOfThought = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory, openAIPromptExecutionSettings))
{
    decisionResponseChainOfThought += content;
    Console.Write(content);
}

### Step 1: Understand the Problem
The farmer needs to transport a wolf, a goat, and a cabbage across the river using a boat that can only carry one item at a time. However, there are constraints:
1. The wolf will eat the goat if left alone together.
2. The goat will eat the cabbage if left alone together.
The goal is to figure out how the farmer can safely transport all three items across the river without any of them being eaten.

### Step 2: Break Down the Reasoning Process
To solve this problem, we need to:
1. Identify the constraints and ensure they are not violated at any point.
2. Develop a step-by-step plan to transport all three items across the river safely.
3. Verify that the solution works by checking each step against the constraints.

### Step 3: Explain Each Step
Here’s the logical sequence to solve the problem:

1. **First Trip:** The farmer takes the goat across the river first.  
   - This is because the goat is the "middle" item in terms of relationships: it can be e

The Generative AI model has generated two different decision approaches to the same retirement decision question. An LLM can be used to decide which decision approach is of better quality. The evaluation could be done by another AI model or an AI model from different AI provider to reduce potential bias. 

After running the evaluation of the decision prompts below, notice the GenAI LLM model prefers the approach of the Chain of Thought (CoT) over the simple system prompt instruction. Furthermore, the AI model appreciates using the "Decision Communication" step with the applied Minto Pyramid.  

> Note: When running the decision approach evaluation below, you may notice that sometimes Approach 1 is preferred. This happens as Approach 2 with Chain of Thought is considered verbose and not a clear as Approach 1 to the AI model.  In a real-world implementation, the prompt instruction will have this made very clear on what the criteria are. I specifically left that instruction out of the prompt to illustrate that even an AI model can vary it's preference!  

In [15]:
var systemPromptEvaluateResponses = """
You are an assistant that is evaluating an approach response to a question.
You will be provided with an important decision as well as two proposed approaches.
The two approaches are labeled "Approach 1" and "Approach 2".

Compare the two approaches and evaluate them based on their: 
logical reasoning of approach, detail of the approach to the decision and the quality the reasoning communicated. 

Create a final score between 1 and 10 for each approach based on the evaluation criteria.
""";

var decisionEvaluationTemplateApproaches = $"""
Decision Scenario: 
{puzzlePrompt}
------------------------------------------------
Approach 1: 
{decisionResponse} 
End of Approach 1
------------------------------------------------
Approach 2: 
{decisionResponseChainOfThought} 
End of Approach 2
""";

// Create a new chat history object using the new Chain of Thought system prompt
var chatHistoryApproachEvaluation = new ChatHistory();
chatHistoryApproachEvaluation.AddSystemMessage(systemPromptEvaluateResponses);
chatHistoryApproachEvaluation.AddUserMessage(decisionEvaluationTemplateApproaches);

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2500, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };


Console.WriteLine("Decision Approaches Evaluation: ");
var evaluationResponseApproach1 = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryApproachEvaluation, openAIPromptExecutionSettings))
{
    evaluationResponseApproach1 += content;
    Console.Write(content);
}

Decision Approaches Evaluation: 
### Evaluation of the Two Approaches:

#### Logical Reasoning of Approach:
- **Approach 1:** The reasoning is clear and follows a step-by-step sequence to solve the problem. It explicitly identifies the constraints (wolf eats goat, goat eats cabbage) and ensures that these constraints are not violated at any step. The logical flow is straightforward and easy to follow.
- **Approach 2:** This approach also provides a clear and logical sequence of steps. It begins with a breakdown of the problem, explicitly stating the constraints and the reasoning behind each action. The explanation is slightly more detailed in terms of justifying why each step is taken, which adds depth to the logical reasoning.

#### Detail of the Approach to the Decision:
- **Approach 1:** The solution is detailed and provides a structured, step-by-step explanation of how to transport the items safely across the river. However, it does not explicitly explain the reasoning behind why e

### Step 3 - Collective Intelligence for Decision Intelligence 

#### Pooling Wisdom of Multiple Opinions 

Imagine you’ve hurt your leg playing a game of football. You have another football season quickly approaching and you would like to understand how long will it take to heal properly. Furthermore, you would like to understand the optimal treatment to ensure you are ready at the start of the football season. You decide to visit three different doctors for their medical opinions. You don’t want to rely on just one doctor because each professional might notice something the others miss. Each specialist may decide to recommend a different approach based on their expertise or their own experience treating leg injuries. After you receive opinions from three different specialists, you think about their recommendations and then formulate a path forward (decision) for a treatment plan for your leg. The final treatment plan could be a result a variety of factors. For example, it could be simple and all three doctors could recommend 4-6 weeks rest. What if the doctors diverge in their opinions? What if 2 of 3 doctors recommend rest and a third recommends an additional procedure on top of rest? What if all three doctors opinions totally diverge? Now you have to judge those doctor opinions collectively or potentially weight one doctor's opinion more significantly. 

The idea of multiple experts arriving at a cohesive conclusion isn’t new. It has been demonstated that pooling the wisdom of multiple “opinions” can often outperform a single prediction. You have probably have heard the terms: Wisdom of the Crowds, Collective Intelligence, Bootstrapping (technique in statistical analysis), Ensembling (Machine Learing) or Mixture of Experts (Generative AI). These are all similar techniques (with their own unique implementations) that derive of the core concept of "pooling wisdom of multiple opinions".  

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SelfConsistency.png">

#### Examples of "Wisdom of the Crowds" at Scale

"Wisdom of the Crowds" can scale well beyond just a few doctor's opinions. In the examples below, note the number of samples collected for each situation is much greater than just a few doctors. In fact, the amount of samples collected can be in the hundreds and the Collective Intelligence power can hold true. 

**Francis Galton’s Ox-Weighing Experiment (1906)** 

Sir Francis Galton, an English statistician, collected 787 guesses from fairgoers trying to estimate the weight of an ox. The average guess was 1,197 pounds, and the actual weight of the ox turned out to be 1,198 pounds. To Galton’s surprise, the average of these guesses (1,197 pounds) came remarkably close to the ox’s actual weight (1,198 pounds)—off by a single pound, or less than 0.1%. That level of accuracy was notable because the crowd was composed of a mixed audience: farmers with relevant experience, but also onlookers, tradespeople, and curious fairgoers with no particular expertise.

**Modern “Jelly Bean Jar” Contests**

Many schools and charities run fundraisers where people pay to guess the number of jelly beans in a jar. It’s commonly observed that while individual guesses can be wildly off, the average of a sufficiently large group is typically within a small percentage (often under 5% error) of the true count. As an example, Michael Mauboussin ran an 2007 experiment where Mauboussin presented a jar of jelly beans to 73 Columbia Business School students. The students' guesses ranged from 250 to 4,100, with an average error of 62%. However, the group's average guess was 1,151, which represented only a 3% off the correct number. Only two students guessed better!

The cool part of the “Jelly Bean Jar” experiment is that it is very approachable to run yourself. A short (5 minute) YouTube video illustrates this below: 
[![Jelly Bean Jar Experiment](https://img.youtube.com/vi/AuQdoAa2FUs/0.jpg)](https://www.youtube.com/watch?v=AuQdoAa2FUs)

**“Ask the Audience” on Who Wants to Be a Millionaire?**

On the TV quiz show, contestants can use the “Ask the Audience” lifeline to poll the studio audience for the correct answer. Historical data shows that the audience collectively identifies the correct answer around 90% of the time—significantly more accurate than individual expert panels, or even the “Phone a Friend” lifeline.

**The U.S. Navy’s Hunt for the Missing Submarine Scorpion (1968)**

When the USS Scorpion, a nuclear-powered submarine, vanished in 1968, the Navy enlisted a broad range of specialists to harness their varied expertise and wisdom for a methodical search. By applying Bayesian statistical techniques to each person’s estimate of where the submarine might lie, they were able to synthesize these disparate perspectives into a single, remarkably accurate prediction. The team’s calculated guess put the submarine’s probable location just a few hundred yards from its actual resting place! This is a testament to the effectiveness of coalescing multiple expert viewpoints. This success with the USS Scorpion search later inspired more widespread use of Bayesian approaches for complex rescue and recovery operations, affirming the power of collective intelligence in high-stakes scenarios.